In [1]:
import numpy as np
import glob
import os 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [34]:
#read csv and create a dataframe
df = pd.read_csv('science_&stem_salaries.csv')

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62642 entries, 0 to 62641
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                62642 non-null  object 
 1   company                  62637 non-null  object 
 2   level                    62523 non-null  object 
 3   title                    62642 non-null  object 
 4   totalyearlycompensation  62642 non-null  int64  
 5   location                 62642 non-null  object 
 6   yearsofexperience        62642 non-null  float64
 7   yearsatcompany           62642 non-null  float64
 8   tag                      61788 non-null  object 
 9   basesalary               62642 non-null  float64
 10  stockgrantvalue          62642 non-null  float64
 11  bonus                    62642 non-null  float64
 12  gender                   43102 non-null  object 
 13  otherdetails             40137 non-null  object 
 14  cityid                

In [ ]:
df.describe()

In [ ]:
# we leave a holdout set, not to open until finishing the modeling
from sklearn.model_selection import train_test_split

df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
df = df[['timestamp','company', 'level', 'title', 'totalyearlycompensation',
       'location', 'yearsofexperience', 'yearsatcompany', 'tag', 'basesalary',
       'stockgrantvalue', 'bonus', 'gender', 'Race', 'Education']]

df.shape # our real dataset consists of 14 features and 1 target variable

In [ ]:
import seaborn as sns 
fig, ax = plt.subplots( nrows = 2, figsize = (18,4) )  

sns.boxplot( df.basesalary, ax = ax[0] )

sns.boxplot( df.totalyearlycompensation, ax = ax[1] ,)

In [ ]:
df.corr()['basesalary']

### "stockgrantvalue" and "totalyearlycompensation" both have high correlation with base salary, but in reality we shouldn't use this columns because they won't be available once we    want to predict for a new position. These columns are invalid for a predictive model so I'll need to remove them

In [ ]:
# I remove very low salary values which could be results of failed scraping, and very high values (2% of outliers)
df = df[df.basesalary > 10 ]
q98 = np.quantile(df.basesalary,0.98)
print(q98)
df.loc[ df.basesalary >q98 , 'basesalary'] =q98

In [ ]:
df.isna().sum()

In [ ]:
# drop rows in columns which NAs are very few and so dropping the whole row doesn't remove that much information from the data
df = df.dropna(subset = ['level', 'tag', 'company'])
df.isna().sum()


In [ ]:
# new variable keeping only date and assigning it a ranking based on chronology
df['datestamp']= df.timestamp.dt.date

df_timestamp = df.datestamp.drop_duplicates().sort_values().reset_index().reset_index()[['level_0','datestamp']]

df = df.merge(df_timestamp).rename(columns = {'level_0': 'date_rank'})

del df['datestamp']
del df['timestamp']

## categorical variables encoding 
https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02

In [ ]:
cat_feats = ['company', 'level', 'title', 'location', 'tag', 'gender', 'Race', 'Education']

In [ ]:
df[cat_feats].describe().T.sort_values('unique')